# Run PyTorchJob From Function

In this Notebook we are going to create [Kubeflow PyTorchJob](https://www.kubeflow.org/docs/components/training/pytorch/).

The PyTorchJob will run distributive training using [DistributedDataParallel strategy](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html).

## Install Kubeflow Python SDKs

You need to install PyTorch packages and Kubeflow SDKs to run this Notebook.

In [ ]:
!pip install torch==1.12.1
!pip install torchvision==0.13.1

# TODO (andreyvelich): Change to release version when SDK with the new APIs is published.
!pip install git+https://github.com/kubeflow/training-operator.git#subdirectory=sdk/python

## Create Train Script for CNN Model

This is simple **Convolutional Neural Network (CNN)** model for recognizing different picture of clothing using [Fashion MNIST Dataset](https://github.com/zalandoresearch/fashion-mnist).

In [2]:
def train_pytorch_model(parameters):
    import logging
    import os

    import torch
    import torch.distributed as dist
    import torch.nn.functional as F
    from torch import nn
    from torch.utils.data import DistributedSampler
    from torchvision import datasets, transforms

    logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s",
        datefmt="%Y-%m-%dT%H:%M:%SZ",
        level=logging.INFO,
    )

    # Create PyTorch CNN Model.
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5, 1)
            self.conv2 = nn.Conv2d(20, 50, 5, 1)
            self.fc1 = nn.Linear(4 * 4 * 50, 500)
            self.fc2 = nn.Linear(500, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 4 * 4 * 50)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    # Get dist parameters.
    # Kubeflow Training Operator automatically set appropriate RANK and WORLD_SIZE based on the configuration.
    RANK = int(os.environ["RANK"])
    WORLD_SIZE = int(os.environ["WORLD_SIZE"])

    # IF GPU is available, nccl dist backend is used. Otherwise, gloo dist backend is used.
    if torch.cuda.is_available():
        device = "cuda"
        backend = "nccl"
    else:
        device = "cpu"
        backend = "gloo"

    logging.info(f"Using Device: {device}, Backend: {backend}")

    model = Net()
    # Attach model to the device.
    model = model.to(device)

    # Attach model to DistributedDataParallel strategy.
    dist.init_process_group(backend="gloo", rank=RANK, world_size=WORLD_SIZE)
    Distributor = nn.parallel.DistributedDataParallel
    model = Distributor(model)

    # Get Fashion MNIST Dataset.
    dataset = datasets.FashionMNIST(
        "./data",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()]),
    )

    # Every PyTorchJob worker gets distributed sampler of dataset.
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=128,
        sampler=DistributedSampler(dataset),
    )

    # Start Training.
    logging.info(f"Start training for RANK: {RANK}. WORLD_SIZE: {WORLD_SIZE}")

    for epoch in range(int(parameters["NUM_EPOCHS"])):
        model.train()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

        for batch_idx, (data, target) in enumerate(train_loader):
            # Attach tensors to the device.
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                logging.info(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tloss={:.4f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    logging.info("Training is finished")

## Run Training Locally in the Notebook

We are going to download Fashion MNIST Dataset and start local training.

In [3]:
# Set dist env variables to run the above training locally on the Notebook.
import os

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "1234"

# Train Model locally in the Notebook.
train_pytorch_model({"NUM_EPOCHS": "1"})

2024-03-05T21:44:47Z INFO     Using Device: cpu, Backend: gloo
2024-03-05T21:44:47Z INFO     Added key: store_based_barrier_key:1 to store for rank: 0
2024-03-05T21:44:47Z INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



...

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

2024-03-05T21:44:54Z INFO     Start training for RANK: 0. WORLD_SIZE: 1


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



2024-03-05T21:44:54Z INFO     Train Epoch: 0 [0/60000 (0%)]	loss=2.3120
2024-03-05T21:44:54Z INFO     Reducer buckets have been rebuilt in this iteration.
2024-03-05T21:44:57Z INFO     Train Epoch: 0 [1280/60000 (2%)]	loss=2.3065
2024-03-05T21:44:59Z INFO     Train Epoch: 0 [2560/60000 (4%)]	loss=2.2853
2024-03-05T21:45:02Z INFO     Train Epoch: 0 [3840/60000 (6%)]	loss=2.2836
2024-03-05T21:45:05Z INFO     Train Epoch: 0 [5120/60000 (9%)]	loss=2.2772
2024-03-05T21:45:07Z INFO     Train Epoch: 0 [6400/60000 (11%)]	loss=2.2551
2024-03-05T21:45:10Z INFO     Train Epoch: 0 [7680/60000 (13%)]	loss=2.2426
2024-03-05T21:45:13Z INFO     Train Epoch: 0 [8960/60000 (15%)]	loss=2.2236
2024-03-05T21:45:16Z INFO     Train Epoch: 0 [10240/60000 (17%)]	loss=2.1883
2024-03-05T21:45:18Z INFO     Train Epoch: 0 [11520/60000 (19%)]	loss=2.1629
2024-03-05T21:45:21Z INFO     Train Epoch: 0 [12800/60000 (21%)]	loss=2.0908
2024-03-05T21:45:23Z INFO     Train Epoch: 0 [14080/60000 (23%)]	loss=2.0023
2024-03-0

## Start Distributive Training with PyTorchJob

Before creating PyTorchJob, you have to create `TrainingClient()`. It uses [Kubernetes Python client](https://github.com/kubernetes-client/python) to communicate with Kubernetes API server. You can set path and context for [the kubeconfig file](https://kubernetes.io/docs/concepts/configuration/organize-cluster-access-kubeconfig/). The default location for the kubeconfig is `~/.kube/config`.

Kubeflow Training Operator automatically set the appropriate env variables (`MASTER_PORT`, `MASTER_ADDR`, `WORLD_SIZE`, `RANK`) for each PyTorchJob container.

PyTorchJob will train model on 3 epochs with 3 PyTorch workers.

In [4]:
from kubeflow.training import TrainingClient, constants

# Start PyTorchJob Training.
pytorchjob_name = "train-pytorch"

# Since we set `job_kind = PyTorchJob` APIs are going to use PyTorchJob as a default Job kind.
training_client = TrainingClient(job_kind=constants.PYTORCHJOB_KIND)

training_client.create_job(
    name=pytorchjob_name,
    train_func=train_pytorch_model,
    parameters={"NUM_EPOCHS": "3"}, # Input parameters for the train function.
    num_workers=3,  # How many PyTorch Workers will be created.
)

### Check the PyTorchJob Status

Use `TrainingClient()` APIs to get information about created PyTorchJob.

In [5]:
print(f"PyTorchJob Status: {training_client.is_job_running(name=pytorchjob_name)}")

PyTorchJob Status: True


### Get PyTorchJob Pod Names

Since we used 3 workers, PyTorchJob will create 1 master pod and 2 worker pods to execute distributed training.

In [6]:
training_client.get_job_pod_names(pytorchjob_name)

['train-pytorch-master-0', 'train-pytorch-worker-0', 'train-pytorch-worker-1']

### Get PyTorchJob Training Logs

We can get the logs from the master pod.

Every worker processes 20000 data samples on each epoch since we distribute 60000 samples across 3 workers.

In [8]:
logs, _ = training_client.get_job_logs(pytorchjob_name)

print(logs["train-pytorch-master-0"])

2024-03-05T21:47:03Z INFO     Using Device: cpu, Backend: gloo
100%|██████████| 26421880/26421880 [00:02<00:00, 12643822.46it/s]
Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
100%|██████████| 29515/29515 [00:00<00:00, 209382.13it/s]

100%|██████████| 4422102/4422102 [00:00<00:00, 5191230.97it/s]
Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw

100%|██████████| 5148/5148 [00:00<00:00, 57733360.94it/s]
Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

2024-03-05T21:47:16Z INFO     Start training for RANK: 0. WORLD_SIZE: 3
2024-03-05T21:47:17Z INFO     Train Epoch: 0 [0/60000 (0%)]	loss=2.2958
2024-03-05T21:47:21Z INFO     Train Epoch: 0 [1280/60000 (6%)]	loss=2.2902
2024-03-05T21:47:26Z INFO     Train Epoch: 0 [2560/60000 (13%)]	loss=2.2863
2024-03-05T21:47:29Z INFO     Train Epo

## Delete PyTorchJob

When PyTorchJob is finished, you can delete the resource.

In [9]:
training_client.delete_job(pytorchjob_name)